In [1]:
import numpy as np
import pandas as pd 
from glob import glob
from os import path
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from torch.autograd import Variable
import torch.utils.data as utils

In [66]:
N = 1200
batch_size = 32
seq_len = 20
num_features = 1
window_size = 10

In [67]:
int(0.8*1200)%32

0

In [68]:
def generate_data(N, seq_len):
    N_train = int(N*0.8)
    X = np.zeros((N, seq_len), dtype=np.float32)
    y = np.zeros((N,1), dtype=np.float32)
    
    indices = np.random.randint(0, N, size=N)
    
    X[indices, 0] = 1.0
    y[indices, 0] = 1.0
    
    X_train, y_train = X[:N_train], y[:N_train]
    X_test, y_test = X[N_train:], y[N_train:]
    
    return X_train, y_train, X_test, y_test
    

In [69]:
def generate_synthetic_test_data(N, seq_len):
    N_train = int(N*0.8)
    X = np.reshape(np.arange(0, N*seq_len, dtype=np.float32),(N, seq_len))
    y = np.reshape(np.arange(0,N, dtype=np.float32), (N,1))
    
    
    X_train, y_train = X[:N_train], y[:N_train]
    X_test, y_test = X[N_train:], y[N_train:]
    
    return X_train, y_train, X_test, y_test

In [70]:
X_train, y_train, X_test, y_test = generate_synthetic_test_data(N, seq_len)

In [170]:
class StatefulSequenceDataset(object):
    def __init__(self, X, y, batch_size, seq_len, window_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.window_size = window_size
        N = X.shape[0]
        
        assert(N%batch_size == 0)
        assert(seq_len > window_size)
        self.N = N
        
        # index into X,y matrices, ranges from 0:(N - batch_size)
        self.idx = 0
        
        # index into subsequence, ranges from 0:window_size
        self.subseq_idx = 0
        
    
    def get_batch(self):
        if self.subseq_idx > self.num_subsequences_per_batch():
            self.idx = (self.idx + self.batch_size) % self.N
            self.subseq_idx = 0
        
        ii = self.idx
        bs = self.batch_size
        
        jj = self.subseq_idx
        batch_X = torch.from_numpy(self.X[ii:ii+bs, jj:jj+self.window_size])
        batch_y = torch.from_numpy(self.y[ii:ii+bs])
        
        self.subseq_idx += 1
        return batch_X, batch_y
    
    def num_batches_per_epoch(self):
        return self.N//self.batch_size
    
    def num_subsequences_per_batch(self):
        return self.seq_len - self.window_size
    
    

In [188]:
ds = StatefulSequenceDataset(X_train, y_train, batch_size, seq_len, window_size)


In [189]:

for x in range(ds.num_batches_per_epoch()):
        for j in range(ds.num_subsequences_per_batch()):
            xx, yy = ds.get_batch()



0
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
